In [1]:
import pandas as pd
import plotly.graph_objects as go

# from paf_calculations import get_daily_rr

wdir = 'X:\\user\\liprandicn\\Health Impacts Model'

In [2]:
# Open excel created with data from Zhao et al. 2021
zhao_df = pd.read_excel(f'{wdir}\\analysis\\zhao2021_tab1.xlsx', header=[0,1], index_col=0)
regions_class = pd.read_excel(f'{wdir}\\data\\IMAGE_regions\\regions_comparison.xlsx')

# Load GBD mortality data
gbd_df = pd.read_csv(f'{wdir}\\data\\GBD_data\\Mortality\\IHME-GBD_2021_DATA.csv')

# Load Carleton results


In [3]:
# Full list of diseases
diseases = {'ckd':'Chronic kidney disease', 'cvd_cmp':'Cardiomyopathy and myocarditis', 'cvd_htn':'Hypertensive heart disease', 
            'cvd_ihd':'Ischemic heart disease', 'cvd_stroke':'Stroke', 'diabetes':'Diabetes mellitus',
            'inj_animal':'Animal contact', 'inj_disaster':'Exposure to forces of nature', 'inj_drowning':'Drowning', 
            'inj_homicide':'Interpersonal violence', 'inj_mech':'Exposure to mechanical forces', 
            'inj_othunintent':'Other unintentional injuries', 'inj_suicide':'Self-harm', 'inj_trans_other':'Other transport injuries', 
            'inj_trans_road':'Road injuries', 'resp_copd':'Chronic obstructive pulmonary disease', 'lri':'Lower respiratory infections'}

In [39]:
# Load approach 1 results

gbd_ = gbd_df[
    (gbd_df['cause_name'].isin(diseases.values())) &
    (gbd_df['sex_name'] == 'Both') &
    (gbd_df['year'].isin(range(2000, 2020))) &
    (gbd_df['age_name'] == 'All ages') &
    (gbd_df['location_name'] != 'Global')][['location_id', 'location_name', 'cause_name', 'year', 'val', 'upper', 'lower']]	
gbd_['year']=gbd_['year'].astype(int)

app1 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_1erf_2000-2019.csv', header=[0,1,2], index_col=[0])
app2 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_2000-2019.csv', header=[0,1,2], index_col=[0])
app3 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_extrap_2000-2019.csv', header=[0,1,2], index_col=[0])
app4 = pd.read_csv(f'{wdir}\\output\\paf_era5_GBD_level3_all-dis_1erf_extrap_2000-2019.csv', header=[0,1,2], index_col=[0])

def process_app(app):
    
    # Select deaths from both heat and cold
    app = app.xs('all', level=2, axis=1)
    # Stack dataframe to have location, year, cause_key as index and paf as column
    app = app.stack(level=[0,1], future_stack=True).reset_index().rename(columns={'level_0':'location_id', 'level_1':'year', 'level_2':'cause_key', 0:'paf'})
    # Map diseases names with keys
    app['cause_name'] = app['cause_key'].map({k:v for k,v in diseases.items()})
    app['year'] = app['year'].astype(int)
    # Merge with mortality data
    app = app.merge(gbd_, on=['location_id', 'cause_name', 'year'], how='left')
    # Calculate deaths attributable to temperature
    app['mean'] = app['val'] * app['paf'] 
    app['upper_est'] = app['upper'] * app['paf'] 
    app['lower_est'] = app['lower'] * app['paf'] 
    app = app[['year', 'cause_name', 'location_name', 'mean', 'upper_est', 'lower_est']]  
    # Merge with regions classification
    app = app.merge(regions_class[['gbd_level3_mor', 'continents']], left_on='location_name', right_on='gbd_level3_mor', how='left')
    app = app.groupby(['year','continents']).sum().reset_index()[['continents', 'year', 'mean', 'upper_est', 'lower_est']]
    app = app.groupby(['continents']).mean().reset_index()
    
    return app

app1 = process_app(app1)
app2 = process_app(app2)
app3 = process_app(app3)
app4 = process_app(app4)

In [44]:
carleton_df = pd.read_csv(f'X:\\user\\liprandicn\\mt-comparison\\Carleton2022\\output\\mortality_ERA5_gbd-level3_2000-2019.csv', index_col=[0,1,2,3])
carleton_df = carleton_df.xs('SSP2', level=0).xs('All', level=1).reset_index().groupby('gbd_level3').sum().iloc[:,1:]#.mean(axis=1)
carleton_df = regions_class.set_index('gbd_level3_mor').merge(carleton_df, left_index=True, right_index=True).reset_index().iloc[:,5:].groupby('continents').mean().mean(axis=1).reset_index()
carleton_df.rename(columns={0:'mean'}, inplace=True)
carleton_df['upper_est'] = carleton_df['mean'] 
carleton_df['lower_est'] = carleton_df['mean']

In [ ]:
df0 = zhao_df.xs('overall', axis=1, level=0).iloc[1:,:]
df1 = app1.set_index('continents').loc[df0.index]
df2 = app2.set_index('continents').loc[df0.index]
df3 = app3.set_index('continents').loc[df0.index]
# df4 = app4.set_index('continents').loc[df0.index]
df5 = carleton_df.set_index('continents').loc[df0.index]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df0.index,
    y=df0["mean"],
    mode="markers",
    name = 'Zhao et al. 2021',
    error_y=dict(
        type="data",
        symmetric=False,
        array=df0["p95"] - df0["mean"],
        arrayminus=df0["mean"] - df0["p5"]
    ),
    marker=dict(size=10, color="cadetblue")
))

dfs = [df1, df2, df3, df5]#, df4]
names = ['GBD (1ERF)', 'GBD', 'GBD (ext)', 'Carleton']#, 'GBD (1ERF & ext)']
colors = ['lightcoral', 'sienna', 'rosybrown', 'grey']#, 'darkorange']

for df, name, color in zip(dfs, names, colors):

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["mean"],
        mode="markers",
        name = name,
        error_y=dict(
            type="data",
            symmetric=False,
            array=df["upper_est"] - df["mean"],
            arrayminus=df["mean"] - df["lower_est"]
        ),
        marker=dict(size=10, color=color)
    ))


fig.update_layout(
    title="Estimation including 5–95% range",
    yaxis_title="Average annual deaths (2000-2019)",
    xaxis_title="Continent",
    height=500,
    width=700,
)

fig.show()